In [18]:
# http://www.shirpeled.com/2018/10/a-hands-on-tutorial-for-zero-knowledge_4.html

import random

def get_witness(problem, assignment):
    """
    Given an instance of a partition problem via a list of numbers (the problem) and a list of
    (-1, 1), we say that the assignment satisfies the problem if their dot product is 0.
    """
    assert len(problem) == len(assignment)

    # partial sum
    s = 0
    side_obfuscator = 1 if random.random() > 0.5 else -1
    witness = [s]
    
    for num, side in zip(problem, assignment):
        assert side == 1 or side == -1
        s += side * num * side_obfuscator
        witness.append(s)
    
    assert s == 0
    shift = random.randint(0, max(0, max(problem)))
    witness = [x + shift for x in witness]
        
    return witness

### Test
problem = [4, 11, 8, 1]
assignment = [1, -1, 1, -1]

w = get_witness(problem, assignment)
n = len(problem)

print("problem", problem)
print("witness", w)

### Checks
assert w[0] == w[n]

for i in range(n):
    assert abs(w[i + 1] - w[i]) == abs(problem[i])


problem [4, 11, 8, 1]
witness [2, 6, -5, 3, 2]


In [19]:
import hashlib
from math import log2, ceil

def hash_string(s):
    return hashlib.sha256(s.encode()).hexdigest()

class MerkleTree:
    """
    A naive Merkle tree implementation using SHA256
    """
    def __init__(self, data):
        self.data = data
        next_pow_of_2 = int(2**ceil(log2(len(data))))
        self.data.extend([0] * (next_pow_of_2 - len(data)))
        self.tree = ["" for x in self.data] + \
                    [hash_string(str(x)) for x in self.data]
        for i in range(len(self.data) - 1, 0, -1):
            self.tree[i] = hash_string(self.tree[i * 2] + self.tree[i * 2 + 1])

    def get_root(self):
        return self.tree[1]

    def get_val_and_path(self, i):
        val = self.data[i]
        auth_path = []
        i = i + len(self.data)
        while i > 1:
            auth_path += [self.tree[i ^ 1]]
            i = i // 2
        return val, auth_path

def verify(root, data_size, i, value, path):
    cur = hash_string(str(value))
    tree_node_id = i + int(2**ceil(log2(data_size)))
    for sibling in path:
        assert tree_node_id > 1
        if tree_node_id % 2 == 0:
            cur = hash_string(cur + sibling)
        else:
            cur = hash_string(sibling + cur)
        tree_node_id = tree_node_id // 2
    assert tree_node_id == 1
    return root == cur

data = ["Yes", "Sir", "I Can", "Boogie!"]
merkle_tree = MerkleTree(data)

root = merkle_tree.get_root()
(val, path) = merkle_tree.get_val_and_path(1)

verify(root, len(data), 1, val, path)

True

In [20]:
import random

class ZkMerkleTree:
    """
    A Zero Knowledge Merkle tree implementation using SHA256
    """
    def __init__(self, data):
        self.data = data
        next_pow_of_2 = int(2**ceil(log2(len(data))))
        self.data.extend([0] * (next_pow_of_2 - len(data)))
        # Intertwine with randomness to obtain zero knowledge.
        rand_list = [random.randint(0, 1 << 32) for x in self.data]
        self.data = [x for tup in zip(self.data, rand_list) for x in tup]
        # Create bottom level of the tree (i.e. leaves).
        self.tree = ["" for x in self.data] + \
                    [hash_string(str(x)) for x in self.data]
        for i in range(len(self.data) - 1, 0, -1):
            self.tree[i] = hash_string(self.tree[i * 2] + self.tree[i * 2 + 1])

    def get_root(self):
        return self.tree[1]

    def get_val_and_path(self, i):
        # Because of the zk padding, the data is now at id * 2
        i = i * 2
        val = self.data[i]
        auth_path = []
        i = i + len(self.data)
        while i > 1:
            auth_path += [self.tree[i ^ 1]]
            i = i // 2
        return val, auth_path

def verify_zk(root, data_size, i, value, path):
    cur = hash_string(str(value))
    # Due to zk padding, data_size needs to be multiplied by 2, as does the value_id
    tree_node_id = i * 2 + int(2**ceil(log2(data_size * 2)))
    for sibling in path:
        assert tree_node_id > 1
        if tree_node_id % 2 == 0:
            cur = hash_string(cur + sibling)
        else:
            cur = hash_string(sibling + cur)
        tree_node_id = tree_node_id // 2
    assert tree_node_id == 1
    return root == cur

data = ["Yes", "Sir", "I Can", "Boogie!"]
zk_merkle_tree = ZkMerkleTree(data)

root = zk_merkle_tree.get_root()
(val, path) = zk_merkle_tree.get_val_and_path(1)

verify_zk(root, len(data), 1, val, path)

True

In [21]:
def get_proof(problem, assignment, num_queries):
    proof = []
    randomness_seed = problem[:]
    for i in range(num_queries):
        witness = get_witness(problem, assignment)
        tree = ZkMerkleTree(witness)
        random.seed(str(randomness_seed))
        query_idx = random.randint(0, len(problem))
        query_and_response = [tree.get_root()]
        query_and_response += [query_idx]
        query_and_response += tree.get_val_and_path(query_idx)
        query_and_response += tree.get_val_and_path((query_idx + 1) % len(witness))
        proof += [query_and_response]
        randomness_seed += [query_and_response]
    return proof

def verify_proof(problem, proof):
    proof_checks_out = True
    randomness_seed = problem[:]
    for query in proof:
        random.seed(str(randomness_seed))
        query_idx = random.randint(0, len(problem))
        merkle_root = query[0]
        proof_checks_out &= query_idx == query[1]
        # Test witness properties.
        if query_idx < len(problem):
            proof_checks_out &= abs(query[2] - query[4]) == abs(problem[query_idx])
        else:
            proof_checks_out &= query[2] == query[4]
        # Authenticate paths
        proof_checks_out &= \
            verify_zk_merkle_path(merkle_root, len(problem) + 1, query_idx, query[2], query[3])
        proof_checks_out &= \
            verify_zk_merkle_path(merkle_root, len(problem) + 1, \
                                 (query_idx + 1) % (len(problem) + 1), query[4], query[5])
        randomness_seed += [query]
    return proof_checks_out

def test(q):
    problem = [1, 2, 3, 6, 6, 6, 12]
    assignment = [1, 1, 1, -1, -1, -1, 1]
    proof = get_proof(problem, assignment, q)
    print(proof)
    return verify_proof(problem, proof)

test(4)

[['bda6731996768a7c7b22e15f9b5c471e24f9f62280d9e00850fa0a0419916ab5', 3, -2, ['a47072bd2b6331c7a62342461ac054707b9fa7d9a9ab89a3a5f87791fa1bb462', 'bfe8106e3a1c642d67a77291460dced46d3f02715c0c7a7a3d00f3dfafd82abb', '4fadeb086017522850e824628168b54d4ca51d0dc58924fc08c82a97645cfe3d', '31cdd36cbead976c90e746f9c32230d5a432c476396774bde5f73058b45a0c16'], 4, ['6df2a639d10bc7ccb8f5ecce2b8646aa54b9d45c63feca81033c1ccd5a45f93c', 'a909e6d4ee4b74fa7e485e5a726ce67639052cb9424c86ded82ee0f9429a2b1c', 'b9a296bf77dca655aeed49eb07020b5204ab93b354b4385b38642d1a1df9e25d', '0b08e63cdfd1b758e2627c229e148b641982fba468832825164064c8f3d5a8a7']], ['b45be0a0000bd9cb6e77ffe40e73578ec5d276a22b40f5c2508f056d6c5bec56', 6, 15, ['b703a38bb22b758c5c23c08f096b6c3155c56885d57e1280ff521126282fa857', 'b59f0967fa41b6e5ef76d7adef6dff327d60cb23fcaf54361c93e0b07850d4b4', 'df6350b39f2cdc5e768d4f99b129c5c288c7feafb4132c5346535ecdc9c8ed0c', '21531f0f5e521ba665dee3cfcc45045336e132418975e1c5162f1e204d8e69ba'], 3, ['3eab3273f262d71c

True